In [4]:
import pandas as pd

# Dataset
We use the ECG dataset available [here](https://www.kaggle.com/shayanfazeli/heartbeat/data?select=mitbih_train.csv). There is [an article](https://arxiv.org/pdf/1805.00794.pdf) that uses the dataset, which we can use as a reference. More details about the dataset can be found [here](https://physionet.org/content/apnea-ecg/1.0.0/)

1. Train only on normal cases. --> determine average error.
2. Use the error as the threshold for decisions
3. Check if the error differs significantly for other classes.

## Load the datasets

In [15]:
train = pd.read_csv('data/mitbih_train.csv', low_memory=False, header=None)
train.shape

(87554, 188)

In [19]:
test = pd.read_csv('data/mitbih_test.csv', low_memory=False, header=None)
test.shape

(21892, 188)

In [10]:
normal = pd.read_csv('data/ptbdb_normal.csv',low_memory=False, header=None)
normal.shape

(4045, 188)

In [20]:
abnormal = pd.read_csv('data/ptbdb_abnormal.csv', low_memory=False, header=None)
abnormal.shape

(10506, 188)

## Split into X and y

In [148]:
labels=np.where(test[187]==0)

In [120]:
X_train = train.to_numpy()[:,0:186]
y_train = train.to_numpy()[:,-1].reshape(-1,1)

#X_train= X_train.reshape(-1,1,186)
#X_train.shape

In [152]:
len(labels[0])

18118

## Train

In [94]:
from keras.layers import Dropout
from keras import regularizers, optimizers
from keras.layers import Input, Conv1D, Dense, Flatten, Activation, UpSampling1D, MaxPooling1D, ZeroPadding1D, TimeDistributed
from keras.models import Model, load_model
from keras.layers.core import Reshape

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
import numpy as np

In [133]:
def LSTM_AE(input_shape):
        
    inputs = Input(shape=input_shape)
    encoded = LSTM(32, activation='relu', return_sequences=True)(inputs)
    encoded = LSTM(16, activation='relu', return_sequences=False)(encoded)
        
    decoded = RepeatVector(input_shape[0])(encoded)
    decoded = LSTM(16, activation='relu', return_sequences=True)(decoded)
    decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(input_shape[1]))(decoded)
        
    sequence_autoencoder = Model(inputs, decoded)
    
    return sequence_autoencoder

In [143]:
model = LSTM_AE((186,1))
model.summary()
model.compile(optimizer='adam', loss='mse', metrics=None)

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 186, 1)]          0         
_________________________________________________________________
lstm_78 (LSTM)               (None, 186, 32)           4352      
_________________________________________________________________
lstm_79 (LSTM)               (None, 16)                3136      
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 186, 16)           0         
_________________________________________________________________
lstm_80 (LSTM)               (None, 186, 16)           2112      
_________________________________________________________________
lstm_81 (LSTM)               (None, 186, 32)           6272      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 186, 1)            33 

In [146]:
X_train2 = X_train.reshape(-1,186,1)
X_train2.shape

(87554, 186, 1)

In [147]:
history = model.fit(X_train2, X_train2, epochs=5, verbose=1, shuffle=False, validation_split=0.05)

Epoch 1/5
1042/2600 [===========>..................] - ETA: 6:18 - loss: 450614001664.0000

KeyboardInterrupt: 

In [59]:
X_train.shape

(87554, 186)

In [101]:
X_train = np.array([[0.5,0.7],[0.2,0.5],[0.5,0.7],[0.2,0.9]]).reshape(2, 2, 2)

In [102]:
X_train.shape

(2, 2, 2)

In [103]:
X_train

array([[[0.5, 0.7],
        [0.2, 0.5]],

       [[0.5, 0.7],
        [0.2, 0.9]]])

In [104]:
model.fit(X_train,X_train)

ValueError: in user code:

    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\keras\engine\training.py:541 train_step  **
        self.trainable_variables)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\p

In [115]:
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

In [116]:
m, _,_ = define_models(186,186, 1)

In [117]:
m.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, None, 186)]  0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, None, 186)]  0                                            
__________________________________________________________________________________________________
lstm_62 (LSTM)                  [(None, 1), (None, 1 752         input_21[0][0]                   
__________________________________________________________________________________________________
lstm_63 (LSTM)                  [(None, None, 1), (N 752         input_22[0][0]                   
                                                                 lstm_62[0][1]             

In [121]:
m.compile()
m.fit(X_train,X_train)

ValueError: in user code:

    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensorflow\python\keras\engine\training.py:531 train_step  **
        y_pred = self(x, training=True)
    c:\users\artem los\.conda\envs\tf_prod\lib\site-packages\tensor